# LMSys Chatbot Arena

Propose, document and defend a solution that can determine which answer is better, or if there is a tie, based on criteria you define..  


**Proposal**: Boosting is a machine learning technique that combines the results of several weak model trainings to create a strong model. The process uses a sequence of interactions, where weights are used for the errors obtained in each training session.

**Example**: XGBoost is a machine learning algorithm based on gradient boosting and using decision trees, where each tree tries to correct the errors of the previous tree.

**Features**: For this problem, we will use features extracted from the texts in our dataset.

In this case, we can use exploratory analysis to establish features add to our classification, such as:
- Size of the texts;
- Words present in the question and answears;
- Difference between answears, etc.

## 1 - Pre-processing

In [1]:
# Librarys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 14,5

import seaborn as sns
sns.set_style = 'whitegrid'

import plotly.express as px
px.defaults.template = "plotly_dark"

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# Classification models
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from gensim.models import Word2Vec
import spacy
nlp = spacy.load('en_core_web_sm')

import re
import string

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# loading datasets
df_train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
df_test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
df_submission = df_test.copy()
sample_example = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')
df_train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [3]:
# Sample example visualization
sample_example.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.333333,0.333333,0.333333
1,211333,0.333333,0.333333,0.333333
2,1233961,0.333333,0.333333,0.333333


In [4]:
# train dataset
df_test.head()

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [5]:
# Dataset's shape
print(f'Train dataset shape: {df_train.shape}')
print(f'Test dataset shape: {df_test.shape}')

Train dataset shape: (57477, 9)
Test dataset shape: (3, 4)


In [6]:
# Dataset info
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              57477 non-null  int64 
 1   model_a         57477 non-null  object
 2   model_b         57477 non-null  object
 3   prompt          57477 non-null  object
 4   response_a      57477 non-null  object
 5   response_b      57477 non-null  object
 6   winner_model_a  57477 non-null  int64 
 7   winner_model_b  57477 non-null  int64 
 8   winner_tie      57477 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 3.9+ MB


In [7]:
# Null values train
df_train.isnull().sum()

id                0
model_a           0
model_b           0
prompt            0
response_a        0
response_b        0
winner_model_a    0
winner_model_b    0
winner_tie        0
dtype: int64

In [8]:
# Null values test
df_test.isnull().sum()

id            0
prompt        0
response_a    0
response_b    0
dtype: int64

In [9]:
# Text Preprocessing 
def preprocess(string):
    strip = string.strip('[]')
    parts = [x.strip('"') for x in strip.split('","')]
    return ''.join(parts)
columns = ['prompt','response_a','response_b']

for colum in df_train[columns]:
    df_train[colum] = df_train[colum].apply(preprocess)
    
for colum in df_test[columns]:
    df_test[colum] = df_test[colum].apply(preprocess)
    
df_train.head(2)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0
1,53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0


In [10]:
# create new columns for data exploration
columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

def class_label(df):
    df['class_label'] = None
    
    for index, row in df.iterrows():
        for col in columns:
            if row[col] == 1:
                df.at[index, 'class_label'] = col
                break

# Call the function on your DataFrame
class_label(df_train)

In [11]:
# Define class number
class_number = {'winner_model_a':0 ,'winner_model_b':1,'winner_tie':2}

df_train['class'] = df_train['class_label'].map(class_number)
df_train.head(2)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_label,class
0,30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,winner_model_a,0
1,53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,winner_model_b,1


In [12]:
# Which model was chosen
def chose_model(row):
    if row['class'] == 0:
        return row['model_a']
    elif row['class'] == 1:
        return row['model_b']
    else:
        return 'tie'

df_train['chose_model'] = df_train.apply(chose_model, axis=1)
df_train.head(2)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_label,class,chose_model
0,30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,winner_model_a,0,gpt-4-1106-preview
1,53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,winner_model_b,1,gpt-4-0613


## 2 - Data Exploration

In [13]:
df_train[['winner_model_a','winner_model_b','winner_tie']].mean()

winner_model_a    0.349079
winner_model_b    0.341911
winner_tie        0.309011
dtype: float64

In [14]:
# Analysing class distribuitions
# Objective: is the base balanced?
fig = px.pie(df_train[['winner_model_a','winner_model_b','winner_tie']].mean(), names=df_train['class_label'], title='Class distribuition')
fig.update_layout(width=600, height=400)
fig.show()

In [15]:
# Models distribuitions
# Objective: The winningest models in the comparison
df_models = df_train.groupby('chose_model')['class'].count().reset_index().sort_values(by='class',ascending=False)
df_models.rename(columns={'chose_model':'model'}, inplace=True)

# Results Graphs
fig = px.bar(df_models[1:11].sort_values(by='class'), y='model', x='class', title='Top 10 Most Chosen Models', text_auto='.3s')
fig.update_layout(width=600, height=400)
fig.show()

In [16]:
# Models chosen from those mentioned
# Objective: The most of these models
df_uni1 = df_train.groupby('model_a')['id'].count().reset_index().sort_values('id', ascending=False)
df_uni1.rename(columns={'model_a':'model'}, inplace=True)
df_uni2 = df_train.groupby('model_b')['id'].count().reset_index().sort_values('id', ascending=False)
df_uni2.rename(columns={'model_b':'model'}, inplace=True)

df_uni = df_uni1.merge(df_uni2, how='left', on = 'model')
df_uni['total_mentions'] = df_uni['id_x'] + df_uni['id_y']
mentions = df_uni['total_mentions'].sum()
df_uni['%mentions'] = (df_uni['total_mentions'] / mentions) * 100

df_uni = df_uni.merge(df_models, how='left', on = 'model')
df_uni['chose_in_mentions'] = (df_uni['class'] / df_uni['total_mentions']) * 100
df_uni = df_uni.sort_values(by = 'chose_in_mentions', ascending=False)[:10]
df_uni.head(3)

,model,id_x,id_y,total_mentions,%mentions,class,chose_in_mentions
0,gpt-4-1106-preview,3678,3709,7387,6.426049,4073,55.137404
31,gpt-3.5-turbo-0314,646,656,1302,1.132627,711,54.608295
36,gpt-4-0125-preview,567,593,1160,1.009099,596,51.379310


In [17]:
# Results - Graphs
fig = px.bar(df_uni.sort_values('chose_in_mentions'), y='model', x='chose_in_mentions', title='% Top 10 Most Chosen Models',
             text_auto='.3s')
fig.update_layout(width=600, height=400)
fig.show()

## 3 Modeling

In [18]:
%%time

df = df_train.copy()

# Word Processing
stopwords_list = stopwords.words('english')

def word_processing(text):
    '''Process text: remove special characters, convert to lowercase, tokenize, and remove stopwords'''
    text = re.sub(r'[/<>()|\+\-\$%&#@\'\"]+', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenize the text
    filtered_tokens = [token for token in tokens if token not in stopwords_list]  # Remove stopwords
    return filtered_tokens

# Features engineering
def count_token(tokens):
    '''Count number of tokens'''
    return len(tokens)

def diff_response(tokens1, tokens2):
    '''Difference in token count between responses'''
    len_1 = len(tokens1)
    len_2 = len(tokens2)
    diff = abs(len_1 - len_2)
    return len_1, len_2, diff

def prompt_u_response(text1, text2):
    '''Common words between the prompt and the responses'''
    conj1 = set(text1.split())
    conj2 = set(text2.split())
    intersec = conj1.intersection(conj2)
    return len(intersec)

def aUb(text1, text2):
    '''Common words between the responses'''
    conj1 = set(text1.split())
    conj2 = set(text2.split())
    intersec = conj1.intersection(conj2)
    return len(intersec)

def lexical_diversity(text):
    '''Proportion of unique words'''
    return len(set(text)) / len(text) if text else 0

def avg_words_per_sentence(text):
    '''Calculate average number of words per sentence'''
    sentences = sent_tokenize(text)
    word_count = sum(len(word_tokenize(sentence)) for sentence in sentences)
    return word_count / len(sentences) if sentences else 0

def sentence_diversity(text):
    '''Calculate sentence diversity (variety in sentence length)'''
    sentences = sent_tokenize(text)
    lengths = [len(word_tokenize(sentence)) for sentence in sentences]
    return len(set(lengths)) / len(lengths) if lengths else 0

def Simylarity(text1, text2):
    '''Calculate the similaty between prompt and responses'''
    tokenA = nlp(text1)
    tokenB = nlp(text2)
    return tokenA.similarity(tokenB)

df['similaty_promptUresponse_a'] = df.apply(lambda row: Simylarity(row['prompt'],row['response_a']), axis=1)
df['similaty_promptUresponse_b'] = df.apply(lambda row: Simylarity(row['prompt'],row['response_b']), axis=1)


# Applying word processing to columns
for col in ['prompt', 'response_a', 'response_b']:
    df[col] = df[col].apply(word_processing)
    df[f'{col}_count_token'] = df[col].apply(count_token)

# Applying additional features
for index, row in df.iterrows():
    for col in ['response_a', 'response_b']:
        tokens = row[col]
        text = ' '.join(tokens)
        
        df.at[index, f'{col}_lexical_diversity'] = lexical_diversity(tokens)
        df.at[index, f'{col}_avg_words_per_sentence'] = avg_words_per_sentence(text)
        #df.at[index, f'{col}_keyword_usage'] = keyword_usage(text, keywords)
        df.at[index, f'{col}_sentence_diversity'] = sentence_diversity(text)
        
        # Existing features
        df.at[index, f'{col}_count_token'] = count_token(tokens)

# Applying diferences functions in the train dataframe 
for index, row in df.iterrows():
    response_a_tokens = row['response_a']
    response_b_tokens = row['response_b']
    response_a_text = ' '.join(response_a_tokens)
    response_b_text = ' '.join(response_b_tokens)
    
    len_a, len_b, diff = diff_response(response_a_tokens, response_b_tokens)
    df.at[index, 'response_len_a'] = len_a
    df.at[index, 'response_len_b'] = len_b
    df.at[index, 'response_diff'] = diff
    
    common_words_ab = aUb(response_a_text, response_b_text)
    df.at[index, 'common_words_ab'] = common_words_ab
    
    prompt_tokens = row['prompt']
    prompt_text = ' '.join(prompt_tokens)
    common_words_prompt_a = prompt_u_response(prompt_text, response_a_text)
    common_words_prompt_b = prompt_u_response(prompt_text, response_b_text)
    df.at[index, 'common_words_prompt_a'] = common_words_prompt_a
    df.at[index, 'common_words_prompt_b'] = common_words_prompt_b
    

df.head(2)

CPU times: user 2h 30min 12s, sys: 6.62 s, total: 2h 30min 19s
Wall time: 2h 30min 29s


,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_label,...,response_a_sentence_diversity,response_b_lexical_diversity,response_b_avg_words_per_sentence,response_b_sentence_diversity,response_len_a,response_len_b,response_diff,common_words_ab,common_words_prompt_a,common_words_prompt_b
0,30192,gpt-4-1106-preview,gpt-4-0613,"[morally, right, try, certain, percentage, fem...","[question, whether, morally, right, aim, certa...","[ai, ,, personal, beliefs, opinions, ., howeve...",1,0,0,winner_model_a,...,0.714286,0.679104,12.181818,0.727273,503.0,134.0,369.0,51.0,14.0,6.0
1,53567,koala-13b,gpt-4-0613,"[difference, marriage, license, marriage, cert...","[marriage, license, legal, document, allows, c...","[marriage, license, marriage, certificate, two...",0,1,0,winner_model_b,...,0.846154,0.458886,19.894737,0.736842,305.0,377.0,72.0,55.0,10.0,9.0


In [19]:
df[['similaty_promptUresponse_a','similaty_promptUresponse_b']].describe().T

,count,mean,std,min,25%,50%,75%,max
similaty_promptUresponse_a,57477.0,0.550906,0.204151,-0.204818,0.414306,0.566271,0.702476,1.0
similaty_promptUresponse_b,57477.0,0.550945,0.204965,-0.202276,0.413480,0.566456,0.703588,1.0


In [20]:
%%time

# features and target selection
X = df.drop(['id','model_a','model_b','prompt','response_a','response_b','winner_model_a','winner_model_b','winner_tie',
            'class_label','chose_model','class'], axis=1).values
y = df['class'].values

# Split train and valid
X_train,X_valid,y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

# Models use
models = {
    'XGBoost': XGBClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'CatBoost': CatBoostClassifier(random_state=42,verbose=False)
}

# Trainning the models
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Metrics
result_loss = []
result_models = []

for model_name, model in models.items():
    i = 0
    print(f'Model {model_name}')

    model_losses = []  # Loss Models List
    
    for train_index, valid_index in skf.split(X, y):
        X_train_fold, X_test_fold = X[train_index], X[valid_index]
        y_train_fold, y_test_fold = y[train_index], y[valid_index]
        
        # Training the model
        model.fit(X_train_fold, y_train_fold)
        
        # Metrics evaluation
        y_test_pred_proba = model.predict_proba(X_test_fold)
        loss = metrics.log_loss(y_test_fold, y_test_pred_proba)
        model_losses.append(loss)  # Add loss in list
        
        # Display the results
        print(f'Fold {i} | Log Loss: {loss}')
        i += 1
        
        mean_loss = np.mean(model_losses)
        result_loss.append(model_losses)
        result_models.append((model_name, model_losses))

    print(f'Mean Loss for model {model_name} is {mean_loss}')
    print('---' * 30)

Model XGBoost
Fold 0 | Log Loss: 1.0620611289808153
Fold 1 | Log Loss: 1.0553065486061692
Fold 2 | Log Loss: 1.0504938218092992
Fold 3 | Log Loss: 1.0485339030154668
Fold 4 | Log Loss: 1.0572594613264699
Mean Loss for model XGBoost is 1.054730972747644
------------------------------------------------------------------------------------------
Model GradientBoosting
Fold 0 | Log Loss: 1.0447749652309717
Fold 1 | Log Loss: 1.0437131499066539
Fold 2 | Log Loss: 1.039729726053591
Fold 3 | Log Loss: 1.037531140441299
Fold 4 | Log Loss: 1.0412987054229337
Mean Loss for model GradientBoosting is 1.0414095374110899
------------------------------------------------------------------------------------------
Model CatBoost
Fold 0 | Log Loss: 1.0474826253664862
Fold 1 | Log Loss: 1.0461297317719325
Fold 2 | Log Loss: 1.0392411851408476
Fold 3 | Log Loss: 1.0384787759826692
Fold 4 | Log Loss: 1.0427685913569458
Mean Loss for model CatBoost is 1.0428201819237763
---------------------------------------

In [21]:
# Plot results
data_dict = {'CV': list(range(0, 5))}

for model, values in result_models:
    data_dict[model] = values

results_df = pd.DataFrame(data_dict)

fig = px.line(results_df, x='CV', y=['XGBoost','GradientBoosting','CatBoost'], title='Loss Models')
fig.update_layout(width=600, height=400)
fig.show()

In [22]:
%%time
# The best model are GradientBoosting
# Now, we'll use the gridsearch for tunning hyperparametros
model_grad = GradientBoostingClassifier()
params = {
    'n_estimators':[100, 150, 170],
    'max_depth': [1, 3, 5],
    'learning_rate': [0.1, 0.01]
}

grid = GridSearchCV(model_grad, params, n_jobs=-1, cv=5, scoring='neg_log_loss', verbose=False)
grid.fit(X_train, y_train)

CPU times: user 1min 56s, sys: 4.37 s, total: 2min 1s
Wall time: 39min 19s


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01], 'max_depth': [1, 3, 5],
                         'n_estimators': [100, 150, 170]},
             scoring='neg_log_loss', verbose=False)

In [23]:
print(f'The best params {grid.best_params_}')

The best params {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 170}


In [24]:
# Selection the best model
best_model = grid.best_estimator_

In [25]:
# Avaluation the best model
y_predict_valid_prob = best_model.predict_proba(X_valid)

logloss = metrics.log_loss(y_valid, y_predict_valid_prob)

print(f'LogLoss the Best Models with GridSearch: {logloss}')

LogLoss the Best Models with GridSearch: 1.035106582478063


There was no significant improvement with GridSearch

In [26]:
# Features importance
features_importance = best_model.feature_importances_
features_names = ['prompt_count_token', 'response_a_count_token',
       'response_b_count_token', 'response_a_lexical_diversity',
       'response_a_avg_words_per_sentence', 'response_a_sentence_diversity',
       'response_b_lexical_diversity', 'response_b_avg_words_per_sentence',
       'response_b_sentence_diversity', 'response_len_a', 'response_len_b',
       'response_diff', 'common_words_ab', 'common_words_prompt_a',
       'common_words_prompt_b','similaty_promptUresponse_a', 'similaty_promptUresponse_b']

fig = px.bar(y=features_names, x=features_importance, color=features_importance, title='Features Importance')
scale = px.colors.sequential.BuGn
fig.update_traces(marker=dict(colorscale=scale))
fig.update_layout(width=800, height=500)
fig.show()

Above we can see the most important features for the model

In [27]:
# Preparing the test data to predict
df_test['similaty_promptUresponse_a'] = df_test.apply(lambda row: Simylarity(row['prompt'],row['response_a']), axis=1)
df_test['similaty_promptUresponse_b'] = df_test.apply(lambda row: Simylarity(row['prompt'],row['response_b']), axis=1)



# Applying word processing to columns
for col in ['prompt', 'response_a', 'response_b']:
    df_test[col] = df_test[col].apply(word_processing)
    df_test[f'{col}_count_token'] = df_test[col].apply(count_token)

# Applying additional features
for index, row in df_test.iterrows():
    for col in ['response_a', 'response_b']:
        tokens = row[col]
        text = ' '.join(tokens)
        
        df_test.at[index, f'{col}_lexical_diversity'] = lexical_diversity(tokens)
        df_test.at[index, f'{col}_avg_words_per_sentence'] = avg_words_per_sentence(text)
        #df.at[index, f'{col}_keyword_usage'] = keyword_usage(text, keywords)
        df_test.at[index, f'{col}_sentence_diversity'] = sentence_diversity(text)
        
        # Existing features
        df_test.at[index, f'{col}_count_token'] = count_token(tokens)


# Applying diferences functions in the test dataframe 
for index, row in df_test.iterrows():
    response_a_tokens = row['response_a']
    response_b_tokens = row['response_b']
    response_a_text = ' '.join(response_a_tokens)
    response_b_text = ' '.join(response_b_tokens)
    
    len_a, len_b, diff = diff_response(response_a_tokens, response_b_tokens)
    df_test.at[index, 'response_len_a'] = len_a
    df_test.at[index, 'response_len_b'] = len_b
    df_test.at[index, 'response_diff'] = diff
    
    common_words_ab = aUb(response_a_text, response_b_text)
    df_test.at[index, 'common_words_ab'] = common_words_ab
    
    prompt_tokens = row['prompt']
    prompt_text = ' '.join(prompt_tokens)
    common_words_prompt_a = prompt_u_response(prompt_text, response_a_text)
    common_words_prompt_b = prompt_u_response(prompt_text, response_b_text)
    df_test.at[index, 'common_words_prompt_a'] = common_words_prompt_a
    df_test.at[index, 'common_words_prompt_b'] = common_words_prompt_b

df_test.drop(columns = ['id','prompt','response_a','response_b'], axis=1, inplace=True)
df_test.head()

,similaty_promptUresponse_a,similaty_promptUresponse_b,prompt_count_token,response_a_count_token,response_b_count_token,response_a_lexical_diversity,response_a_avg_words_per_sentence,response_a_sentence_diversity,response_b_lexical_diversity,response_b_avg_words_per_sentence,response_b_sentence_diversity,response_len_a,response_len_b,response_diff,common_words_ab,common_words_prompt_a,common_words_prompt_b
0,0.759573,0.716489,11,4,12,1.000000,4.000000,1.00,0.833333,6.000000,1.000000,4.0,12.0,8.0,3.0,3.0,6.0
1,0.761477,0.825044,57,137,50,0.700730,22.833333,1.00,0.680000,12.500000,0.750000,137.0,50.0,87.0,9.0,11.0,13.0
2,0.755163,0.649378,20,430,358,0.290698,21.500000,0.75,0.407821,29.833333,0.833333,430.0,358.0,72.0,45.0,16.0,15.0


In [28]:
# Predict proba in test
y_sub_proba = best_model.predict_proba(df_test)
y_sub_proba

array([[0.26967621, 0.26320919, 0.4671146 ],
       [0.50344934, 0.24264836, 0.25390231],
       [0.28642862, 0.33796415, 0.37560723]])

In [29]:
# Submission df
submission = pd.DataFrame({
    'id':df_submission['id'],
    'winner_model_a': y_sub_proba[:, 0],
    'winner_model_b': y_sub_proba[:, 1],
    'winner_tie': y_sub_proba[:, 2]
})

submission.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.269676,0.263209,0.467115
1,211333,0.503449,0.242648,0.253902
2,1233961,0.286429,0.337964,0.375607


In [30]:
submission.to_csv('/kaggle/working/submission.csv', index=False)